<a href="https://colab.research.google.com/github/yagnesh2004/Profile/blob/main/object_detection_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python
!pip install cvlib
!pip install pyTelegramBotAPI


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 44.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for cvlib: filename=cvlib-0.2.7-py3-none-any.whl size=10046368 sha256=e9f3cd4255073eb7b82b4facbddfc3536e9f25e9f73fe21138325ef056d5025f
  Stored in directory: /root/.cache/pip/wheels/9e/a5/d4/fe37b48fe4f4b621ba5e574a991230070f3cc4f02322a01489
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12067 sha256=5a321ddf7dd3252b9d62fcd62b4382dec4a87bf613e85e274e56c03933f39393
  Stored in directory: /root/.cache/pip/wheels/cd/17/e5/765d1a3112ff3978f70223502f6047e06c43a24d7c5f8ff95b
Successfully built cvlib progressbar
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=672d791844f4914e82631f9804daeea9180d7a1a2e3d3394

In [2]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import telebot
import numpy as np
from io import BytesIO

# Replace 'YOUR_BOT_TOKEN' with your actual bot token from BotFather
bot = telebot.TeleBot("6418099122:AAEChotvx7tjMT-HJbxsU5Ls31fi9v6UdaY")

@bot.message_handler(content_types=['video'])
def handle_video(message):
    try:
        # Get the video file from the message
        video = bot.get_file(message.video.file_id)
        video_content = bot.download_file(video.file_path)

        # Convert the video content to a numpy array
        nparr = np.frombuffer(video_content, np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)

        # Perform object detection on the frame
        bbox, label, conf = cv.detect_common_objects(frame)
        output_frame = draw_bbox(frame, bbox, label, conf)

        # Save the output frame as a video file
        out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame.shape[1], frame.shape[0]))
        out.write(output_frame)
        out.release()

        # Send the output video back to the user
        with open('output_video.avi', 'rb') as video_file:
            bot.send_video(message.chat.id, video_file)

    except Exception as e:
        bot.reply_to(message, f"Error: {e}")

if __name__ == "__main__":
    bot.polling()
